In [ ]:
import pandas as pd
import glob
import os

# 1. 파일 경로 지정
folder_path = "data\세종도시교통공사_어울링_이용현황"
file_list = glob.glob(os.path.join(folder_path, "*.csv*"))  # .xlsx 또는 .xls 모두 포함
print("탐색된 파일:", file_list)

# 2. 병합
dfs = []
for file in file_list:
    df = pd.read_csv(file, encoding='cp949')
    print(len(df))
    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)

# 3. 결과 확인
print(f"총 {len(merged_df)}행 병합 완료")
print(merged_df.head())

merged_df.to_csv("data/세종도시교통공사_공공자전거(어울링) 이용 현황.csv", index=False, encoding='utf-8-sig')

탐색된 파일: ['data\\세종도시교통공사_어울링_이용현황\\~$세종도시교통공사_공공자전거(어울링) 이용 현황 2023년 1분기.csv', 'data\\세종도시교통공사_어울링_이용현황\\~$세종도시교통공사_공공자전거(어울링) 이용 현황 2023년 2분기.csv', 'data\\세종도시교통공사_어울링_이용현황\\세종도시교통공사_공공자전거(어울링) 이용 현황 2023년 1분기.csv', 'data\\세종도시교통공사_어울링_이용현황\\세종도시교통공사_공공자전거(어울링) 이용 현황 2023년 2분기.csv', 'data\\세종도시교통공사_어울링_이용현황\\세종도시교통공사_공공자전거(어울링) 이용 현황 2023년 3분기.csv', 'data\\세종도시교통공사_어울링_이용현황\\세종도시교통공사_공공자전거(어울링) 이용 현황 2023년 4분기.csv', 'data\\세종도시교통공사_어울링_이용현황\\세종도시교통공사_공공자전거(어울링) 이용 현황 2024년 1분기.csv', 'data\\세종도시교통공사_어울링_이용현황\\세종도시교통공사_공공자전거(어울링) 이용 현황 2024년 2분기.csv']
0
0
440619
791415
629805
591288
398766
830920
총 3682813행 병합 완료
     H         순번         자전거고유번호    시작 대여소                 대여시간    반납 대여소  \
0  NaN  5792004.0  001_0110_02091  SJ_00273  2023-01-01 00:07:55  SJ_00369   
1  NaN  5792005.0  001_0110_01628  SJ_00244  2023-01-01 00:08:00  SJ_00400   
2  NaN  5792006.0  001_0110_02899  SJ_00146  2023-01-01 00:08:07  SJ_00048   
3  NaN  5792007.0  001_0110_02625  SJ_00580  2023-01-01 00:09:42  SJ

In [ ]:
import pandas as pd
import numpy as np
from pulp import LpProblem, LpMaximize, LpVariable, lpSum
from glob import glob

# [1] 대여소 위치 및 kde 데이터 로드
station_df = pd.read_csv("result/어울링_대여소_with_kde.csv", encoding='utf-8-sig')
station_df['대여소 아이디'] = station_df['대여소 아이디'].astype(str).str.replace('SJ_', 'SJ-', regex=False)
location_df = station_df[['대여소 아이디', '위도', '경도', 'kde_density_norm']].dropna()

# [2] 이용현황 파일 병합
usage_file_list = glob("data/세종도시교통공사_공공자전거(어울링) 이용 현황.csv")
usage_dfs = []

for file in usage_file_list:
    df = pd.read_csv(file, encoding='utf-8')
    df['시작 대여소'] = df['시작 대여소'].astype(str).str.replace('SJ_', 'SJ-', regex=False)
    df = df.rename(columns={'시작 대여소': '대여소 아이디'})
    usage_dfs.append(df[['대여소 아이디']])

usage_df = pd.concat(usage_dfs, ignore_index=True)

# [3] 수요 집계
demand_df = usage_df['대여소 아이디'].value_counts().reset_index()
demand_df.columns = ['대여소 아이디', '수요']

# [4] 수요 + kde + 위치 결합
merged_df = pd.merge(demand_df, location_df, on='대여소 아이디')
demand_ids = merged_df['대여소 아이디'].tolist()
candidate_ids = location_df['대여소 아이디'].tolist()

merged_df['수요_norm'] = (merged_df['수요'] - merged_df['수요'].min()) / (merged_df['수요'].max() - merged_df['수요'].min())
merged_df['중요도'] = 0.5 * merged_df['수요_norm'] + 0.5 * merged_df['kde_density_norm']
importance_dict = merged_df.set_index('대여소 아이디')['중요도'].to_dict()

# [5] 벡터화된 커버리지 행렬 계산 (Haversine)
coverage_radius_km = 0.5
demand_coords = location_df[location_df['대여소 아이디'].isin(demand_ids)].copy()
candidate_coords = location_df[location_df['대여소 아이디'].isin(candidate_ids)].copy()

d_lat = np.radians(demand_coords['위도'].values.reshape(-1, 1))
d_lon = np.radians(demand_coords['경도'].values.reshape(-1, 1))
c_lat = np.radians(candidate_coords['위도'].values.reshape(1, -1))
c_lon = np.radians(candidate_coords['경도'].values.reshape(1, -1))

R = 6371.0
dlat = c_lat - d_lat
dlon = c_lon - d_lon
a = np.sin(dlat / 2)**2 + np.cos(d_lat) * np.cos(c_lat) * np.sin(dlon / 2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
dist_matrix = R * c
coverage_bool = dist_matrix <= coverage_radius_km

coverage_matrix = {
    (demand_id, candidate_id): bool(coverage_bool[i, j])
    for i, demand_id in enumerate(demand_coords['대여소 아이디'].values)
    for j, candidate_id in enumerate(candidate_coords['대여소 아이디'].values)
}

# [6] 자동 탐색 MCLP: 목표 커버율 만족하는 최소 p 탐색
target_coverage_ratio = 0.95
total_importance = sum(importance_dict[i] for i in demand_ids)
best_p = None
best_selected_sites = None

for p in range(1, 31):
    model = LpProblem("MCLP", LpMaximize)
    x = LpVariable.dicts("Candidate", candidate_ids, cat='Binary')
    y = LpVariable.dicts("Demand", demand_ids, cat='Binary')

    model += lpSum(y[i] * importance_dict[i] for i in demand_ids)

    for i in demand_ids:
        model += y[i] <= lpSum(x[j] for j in candidate_ids if coverage_matrix.get((i, j), False))
    model += lpSum(x[j] for j in candidate_ids) <= p

    model.solve()

    covered_importance = sum(importance_dict[i] for i in demand_ids if y[i].varValue == 1)
    coverage_ratio = covered_importance / total_importance
    print(f"p = {p}, 커버된 중요도 = {covered_importance:.4f} ({coverage_ratio:.2%})")

    if coverage_ratio >= target_coverage_ratio:
        best_p = p
        best_selected_sites = [j for j in candidate_ids if x[j].varValue == 1]
        break

# [7] 결과 저장
if best_p is not None:
    print(f"\n✅ 최적의 p = {best_p} (커버율 ≥ {target_coverage_ratio:.0%})")
    print("\n📍 최종 선정된 대여소 후보지 ID 목록:")
    for i, site in enumerate(best_selected_sites, 1):
        print(f"{i:2d}. {site}")

    # 저장
    selected_df = location_df[location_df['대여소 아이디'].isin(best_selected_sites)].copy()
    selected_df['선정여부'] = '선정됨'
    selected_df.to_csv(f"result/MCLP_선정_대여소_p{best_p}.csv", index=False, encoding='utf-8-sig')

    merged_df['커버여부'] = merged_df['대여소 아이디'].apply(
        lambda i: int(y[i].varValue) if i in y else 0
    )
    merged_df.to_csv(f"result/MCLP_수요지_커버여부_p{best_p}.csv", index=False, encoding='utf-8-sig')

    print(f"\n💾 결과 저장 완료: result/MCLP_선정_대여소_p{best_p}.csv")
    print(f"💾 결과 저장 완료: result/MCLP_수요지_커버여부_p{best_p}.csv")
else:
    print("❌ 목표 커버율을 만족하는 p를 찾지 못했습니다.")

In [ ]:
# 전체 후보지에 대해 선정여부 표시 (선정: '선정됨', 미선정: '미선정')
full_station_df = location_df.copy()
full_station_df['선정여부'] = full_station_df['대여소 아이디'].apply(
    lambda x: '선정됨' if x in best_selected_sites else '미선정'
)

# 저장
full_station_df.to_csv(f"result/MCLP_전체_대여소_선정여부포함_p{best_p}.csv", index=False, encoding='utf-8-sig')